In [2]:
%pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 9.7 MB/s eta 0:00:00:00:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

2025-09-12 17:55:04.028703: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-12 17:55:04.638253: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-12 17:55:06.275600: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
animes_df = pd.read_csv("data/animes.csv")
ratings_df = pd.read_csv("data/reviews.csv")
profiles_df = pd.read_csv("data/profiles.csv")

In [5]:
# See the first 5 rows
print(animes_df.head())
print(ratings_df.head())
print(profiles_df.head())

     uid                             title  \
0  28891           Haikyuu!! Second Season   
1  23273           Shigatsu wa Kimi no Uso   
2  34599                     Made in Abyss   
3   5114  Fullmetal Alchemist: Brotherhood   
4  31758  Kizumonogatari III: Reiketsu-hen   

                                            synopsis  \
0  Following their participation at the Inter-Hig...   
1  Music accompanies the path of the human metron...   
2  The Abyss—a gaping chasm stretching down into ...   
3  "In order for something to be obtained, someth...   
4  After helping revive the legendary vampire Kis...   

                                               genre  \
0  ['Comedy', 'Sports', 'Drama', 'School', 'Shoun...   
1  ['Drama', 'Music', 'Romance', 'School', 'Shoun...   
2  ['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...   
3  ['Action', 'Military', 'Adventure', 'Comedy', ...   
4   ['Action', 'Mystery', 'Supernatural', 'Vampire']   

                          aired  episodes  member

In [16]:
# Extract start year from aired
animes_df["year"] = animes_df["aired"].str.extract(r'(\d{4})').astype(float)

# One-hot encode genres
genre_dummies = animes_df["genre"].str.join('|').str.get_dummies()

# Select numeric features
num_features = animes_df[["year", "score"]].fillna(0)

# Normalize numeric features
scaler = MinMaxScaler()
num_features = pd.DataFrame(
    scaler.fit_transform(num_features),
    columns=num_features.columns,
    index=animes_df.index
)

# Combine everything
item_train = pd.concat([num_features, genre_dummies], axis=1)
print(item_train.head())

       year     score  Action  Adventure  Cars  Comedy  Dementia  Demons  \
0  0.997031  0.955580       0          0     0       1         0       0   
1  0.996536  0.956663       0          0     0       0         0       0   
2  0.998021  0.956663       0          1     0       0         0       0   
3  0.994062  1.000000       1          1     0       1         0       0   
4  0.998021  0.956663       1          0     0       0         0       0   

   Drama  Ecchi  ...  Shounen Ai  Slice of Life  Space  Sports  Super Power  \
0      1      0  ...           0              0      0       1            0   
1      1      0  ...           0              0      0       0            0   
2      1      0  ...           0              0      0       0            0   
3      1      0  ...           0              0      0       0            0   
4      0      0  ...           0              0      0       0            0   

   Supernatural  Thriller  Vampire  Yaoi  Yuri  
0             0    

In [20]:
## --- User features ---
import ast

# Convert stringified lists to real lists of ints
def parse_favorites(x):
    try:
        return [int(a) for a in ast.literal_eval(x)]
    except:
        return []  # fallback for malformed entries

profiles_df["favorites_anime"] = profiles_df["favorites_anime"].apply(parse_favorites)
# Step 2: Build user_train
user_features = []
# Create a mapping from anime UID to row index in item_train
anime_id_to_idx = {uid: idx for idx, uid in enumerate(animes_df["uid"])}

for _, row in profiles_df.iterrows():
    favs = row["favorites_anime"]
    
    # Get indices of these anime in item_train
    indices = [anime_id_to_idx[uid] for uid in favs if uid in anime_id_to_idx]
    
    if indices:
        # Average features of the anime to get user preference vector
        user_vec = item_train.iloc[indices].mean(axis=0)
    else:
        # If no valid favorites, use zero vector
        user_vec = np.zeros(item_train.shape[1])
    
    user_features.append(user_vec)

# Create user_train DataFrame
user_train = pd.DataFrame(user_features, index=profiles_df["profile"])
print(user_train.iloc[0].to_dict())


{'year': 0.9946066303809996, 'score': 0.8813109425785483, 'Action': 0.4, 'Adventure': 0.4, 'Cars': 0.0, 'Comedy': 0.45, 'Dementia': 0.0, 'Demons': 0.05, 'Drama': 0.45, 'Ecchi': 0.0, 'Fantasy': 0.25, 'Game': 0.05, 'Harem': 0.05, 'Hentai': 0.0, 'Historical': 0.1, 'Horror': 0.1, 'Josei': 0.05, 'Kids': 0.0, 'Magic': 0.0, 'Martial Arts': 0.05, 'Mecha': 0.05, 'Military': 0.05, 'Music': 0.1, 'Mystery': 0.25, 'Parody': 0.0, 'Police': 0.05, 'Psychological': 0.15, 'Romance': 0.45, 'Samurai': 0.0, 'School': 0.1, 'Sci-Fi': 0.1, 'Seinen': 0.15, 'Shoujo': 0.05, 'Shoujo Ai': 0.0, 'Shounen': 0.3, 'Shounen Ai': 0.0, 'Slice of Life': 0.3, 'Space': 0.0, 'Sports': 0.0, 'Super Power': 0.15, 'Supernatural': 0.4, 'Thriller': 0.0, 'Vampire': 0.05, 'Yaoi': 0.0, 'Yuri': 0.0}


In [ ]:
# scale training data
item_train_unscaled = item_train.copy()
user_train_unscaled = user_train.copy()
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)